<a href="https://colab.research.google.com/github/AnishaVardh/Breast-Cancer-Classification/blob/main/Breast_Cancer_Logistic_Regression_using_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ANISHA V 19MIS1178

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz

In [ ]:
!tar xf spark-3.2.1-bin-hadoop3.2.tgz

In [ ]:
!pip install -q findspark

In [ ]:
!pwd

/content


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

In [ ]:
!ls "/content/spark-3.2.1-bin-hadoop3.2"

bin   data	jars	    LICENSE   NOTICE  R		 RELEASE  yarn
conf  examples	kubernetes  licenses  python  README.md  sbin


In [ ]:
!ls "/usr/lib/jvm/java-8-openjdk-amd64"

ASSEMBLY_EXCEPTION  docs     jre  man	   THIRD_PARTY_README
bin		    include  lib  src.zip


In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
df = df = spark.read.format("csv").load("/content/drive/MyDrive/data.csv", inferSchema=True, header=True)

In [ ]:
display(df)

DataFrame[id: int, diagnosis: string, radius_mean: double, texture_mean: double, perimeter_mean: double, area_mean: double, smoothness_mean: double, compactness_mean: double, concavity_mean: double, concave points_mean: double, symmetry_mean: double, fractal_dimension_mean: double, radius_se: double, texture_se: double, perimeter_se: double, area_se: double, smoothness_se: double, compactness_se: double, concavity_se: double, concave points_se: double, symmetry_se: double, fractal_dimension_se: double, radius_worst: double, texture_worst: double, perimeter_worst: double, area_worst: double, smoothness_worst: double, compactness_worst: double, concavity_worst: double, concave points_worst: double, symmetry_worst: double, fractal_dimension_worst: double, _c32: string]

In [ ]:
df.show()

+--------+---------+-----------+------------+--------------+---------+---------------+----------------+--------------+-------------------+-------------+----------------------+---------+----------+------------+-------+-------------+--------------+------------+-----------------+-----------+--------------------+------------+-------------+---------------+----------+----------------+-----------------+---------------+--------------------+--------------+-----------------------+----+
|      id|diagnosis|radius_mean|texture_mean|perimeter_mean|area_mean|smoothness_mean|compactness_mean|concavity_mean|concave points_mean|symmetry_mean|fractal_dimension_mean|radius_se|texture_se|perimeter_se|area_se|smoothness_se|compactness_se|concavity_se|concave points_se|symmetry_se|fractal_dimension_se|radius_worst|texture_worst|perimeter_worst|area_worst|smoothness_worst|compactness_worst|concavity_worst|concave points_worst|symmetry_worst|fractal_dimension_worst|_c32|
+--------+---------+-----------+------

In [ ]:
df.printSchema

<bound method DataFrame.printSchema of DataFrame[id: int, diagnosis: string, radius_mean: double, texture_mean: double, perimeter_mean: double, area_mean: double, smoothness_mean: double, compactness_mean: double, concavity_mean: double, concave points_mean: double, symmetry_mean: double, fractal_dimension_mean: double, radius_se: double, texture_se: double, perimeter_se: double, area_se: double, smoothness_se: double, compactness_se: double, concavity_se: double, concave points_se: double, symmetry_se: double, fractal_dimension_se: double, radius_worst: double, texture_worst: double, perimeter_worst: double, area_worst: double, smoothness_worst: double, compactness_worst: double, concavity_worst: double, concave points_worst: double, symmetry_worst: double, fractal_dimension_worst: double, _c32: string]>

 Missing Records

In [ ]:
from pyspark.sql.functions import isnan, when, count, col

In [ ]:
df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).show()

+---+---------+-----------+------------+--------------+---------+---------------+----------------+--------------+-------------------+-------------+----------------------+---------+----------+------------+-------+-------------+--------------+------------+-----------------+-----------+--------------------+------------+-------------+---------------+----------+----------------+-----------------+---------------+--------------------+--------------+-----------------------+----+
| id|diagnosis|radius_mean|texture_mean|perimeter_mean|area_mean|smoothness_mean|compactness_mean|concavity_mean|concave points_mean|symmetry_mean|fractal_dimension_mean|radius_se|texture_se|perimeter_se|area_se|smoothness_se|compactness_se|concavity_se|concave points_se|symmetry_se|fractal_dimension_se|radius_worst|texture_worst|perimeter_worst|area_worst|smoothness_worst|compactness_worst|concavity_worst|concave points_worst|symmetry_worst|fractal_dimension_worst|_c32|
+---+---------+-----------+------------+--------

In [ ]:
df.na.drop().show(5)

+---+---------+-----------+------------+--------------+---------+---------------+----------------+--------------+-------------------+-------------+----------------------+---------+----------+------------+-------+-------------+--------------+------------+-----------------+-----------+--------------------+------------+-------------+---------------+----------+----------------+-----------------+---------------+--------------------+--------------+-----------------------+----+
| id|diagnosis|radius_mean|texture_mean|perimeter_mean|area_mean|smoothness_mean|compactness_mean|concavity_mean|concave points_mean|symmetry_mean|fractal_dimension_mean|radius_se|texture_se|perimeter_se|area_se|smoothness_se|compactness_se|concavity_se|concave points_se|symmetry_se|fractal_dimension_se|radius_worst|texture_worst|perimeter_worst|area_worst|smoothness_worst|compactness_worst|concavity_worst|concave points_worst|symmetry_worst|fractal_dimension_worst|_c32|
+---+---------+-----------+------------+--------

In [ ]:
print(df.count())

569


Coverting data

In [ ]:
#import libraries
from pyspark.ml.feature import StringIndexer, OneHotEncoder

In [ ]:
df.groupBy('diagnosis').count().show()

+---------+-----+
|diagnosis|count|
+---------+-----+
|        B|  357|
|        M|  212|
+---------+-----+



In [ ]:
class_indexer = StringIndexer(inputCol="diagnosis", outputCol="label")
#Fit and transform the dataframe
df = class_indexer.fit(df).transform(df)
df.show(5)

+--------+---------+-----------+------------+--------------+---------+---------------+----------------+--------------+-------------------+-------------+----------------------+---------+----------+------------+-------+-------------+--------------+------------+-----------------+-----------+--------------------+------------+-------------+---------------+----------+----------------+-----------------+---------------+--------------------+--------------+-----------------------+----+-----+
|      id|diagnosis|radius_mean|texture_mean|perimeter_mean|area_mean|smoothness_mean|compactness_mean|concavity_mean|concave points_mean|symmetry_mean|fractal_dimension_mean|radius_se|texture_se|perimeter_se|area_se|smoothness_se|compactness_se|concavity_se|concave points_se|symmetry_se|fractal_dimension_se|radius_worst|texture_worst|perimeter_worst|area_worst|smoothness_worst|compactness_worst|concavity_worst|concave points_worst|symmetry_worst|fractal_dimension_worst|_c32|label|
+--------+---------+------

In [ ]:
df.select(['diagnosis', 'label']).show(5)

+---------+-----+
|diagnosis|label|
+---------+-----+
|        M|  1.0|
|        M|  1.0|
|        M|  1.0|
|        M|  1.0|
|        M|  1.0|
+---------+-----+
only showing top 5 rows



In [ ]:
def transformColumnsToNumeric(df, inputCol):
    
    #apply StringIndexer to inputCol
    inputCol_indexer = StringIndexer(inputCol = inputCol, outputCol = inputCol + "-index").fit(df)
    df = inputCol_indexer.transform(df)
    
    onehotencoder_vector = OneHotEncoder(inputCol = inputCol + "-index", outputCol = inputCol + "-vector")
    df = onehotencoder_vector.fit(df).transform(df)
    
    return df
    
    pass

In [ ]:
df = transformColumnsToNumeric(df, "radius_mean")
df = transformColumnsToNumeric(df, "texture_mean")
df = transformColumnsToNumeric(df, "perimeter_mean")
df = transformColumnsToNumeric(df, "area_mean")
df = transformColumnsToNumeric(df, "smoothness_mean")
df = transformColumnsToNumeric(df, "compactness_mean")
df = transformColumnsToNumeric(df, "concavity_mean")
df = transformColumnsToNumeric(df, "concave points_mean")
df = transformColumnsToNumeric(df, "symmetry_mean")
df = transformColumnsToNumeric(df, "fractal_dimension_mean")
df.show(5)

+--------+---------+-----------+------------+--------------+---------+---------------+----------------+--------------+-------------------+-------------+----------------------+---------+----------+------------+-------+-------------+--------------+------------+-----------------+-----------+--------------------+------------+-------------+---------------+----------+----------------+-----------------+---------------+--------------------+--------------+-----------------------+----+-----+-----------------+------------------+------------------+-------------------+--------------------+---------------------+---------------+-----------------+---------------------+----------------------+----------------------+-----------------------+--------------------+---------------------+-------------------------+--------------------------+-------------------+--------------------+----------------------------+-----------------------------+
|      id|diagnosis|radius_mean|texture_mean|perimeter_mean|area_mean|sm

Vector assembler

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
df.columns

['id',
 'diagnosis',
 'radius_mean',
 'texture_mean',
 'perimeter_mean',
 'area_mean',
 'smoothness_mean',
 'compactness_mean',
 'concavity_mean',
 'concave points_mean',
 'symmetry_mean',
 'fractal_dimension_mean',
 'radius_se',
 'texture_se',
 'perimeter_se',
 'area_se',
 'smoothness_se',
 'compactness_se',
 'concavity_se',
 'concave points_se',
 'symmetry_se',
 'fractal_dimension_se',
 'radius_worst',
 'texture_worst',
 'perimeter_worst',
 'area_worst',
 'smoothness_worst',
 'compactness_worst',
 'concavity_worst',
 'concave points_worst',
 'symmetry_worst',
 'fractal_dimension_worst',
 '_c32',
 'label',
 'radius_mean-index',
 'radius_mean-vector',
 'texture_mean-index',
 'texture_mean-vector',
 'perimeter_mean-index',
 'perimeter_mean-vector',
 'area_mean-index',
 'area_mean-vector',
 'smoothness_mean-index',
 'smoothness_mean-vector',
 'compactness_mean-index',
 'compactness_mean-vector',
 'concavity_mean-index',
 'concavity_mean-vector',
 'concave points_mean-index',
 'concave po

In [ ]:
inputCols=[
        'radius_mean-vector',
        'texture_mean-vector',
        'perimeter_mean-vector',
        'area_mean-vector',
        'smoothness_mean-vector',
        'compactness_mean-vector',
        'concavity_mean-vector',
        'concave points_mean-vector',
        'symmetry_mean-vector',
        'fractal_dimension_mean-vector']

In [ ]:
df_va = VectorAssembler(inputCols = inputCols, outputCol="features")

In [ ]:
df = df_va.transform(df)

In [ ]:
df.select(inputCols + ["features"] ).show(5)

+------------------+-------------------+---------------------+-----------------+----------------------+-----------------------+---------------------+--------------------------+--------------------+-----------------------------+--------------------+
|radius_mean-vector|texture_mean-vector|perimeter_mean-vector| area_mean-vector|smoothness_mean-vector|compactness_mean-vector|concavity_mean-vector|concave points_mean-vector|symmetry_mean-vector|fractal_dimension_mean-vector|            features|
+------------------+-------------------+---------------------+-----------------+----------------------+-----------------------+---------------------+--------------------------+--------------------+-----------------------------+--------------------+
|  (455,[87],[1.0])|   (478,[81],[1.0])|    (521,[113],[1.0])| (538,[29],[1.0])|     (473,[446],[1.0])|      (536,[531],[1.0])|    (536,[519],[1.0])|          (541,[14],[1.0])|   (431,[416],[1.0])|            (498,[483],[1.0])|(5007,[87,536,104...|
| (4

In [ ]:
df.select(['diagnosis','label']).show(10,False)

+---------+-----+
|diagnosis|label|
+---------+-----+
|M        |1.0  |
|M        |1.0  |
|M        |1.0  |
|M        |1.0  |
|M        |1.0  |
|M        |1.0  |
|M        |1.0  |
|M        |1.0  |
|M        |1.0  |
|M        |1.0  |
+---------+-----+
only showing top 10 rows



In [ ]:
df_transformed = df.select(['features','label'])
df_transformed.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(5007,[87,536,104...|  1.0|
|(5007,[383,690,95...|  1.0|
|(5007,[364,517,94...|  1.0|
|(5007,[135,774,12...|  1.0|
|(5007,[375,592,10...|  1.0|
+--------------------+-----+
only showing top 5 rows



In [ ]:
df_transformed.groupBy('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0|  357|
|  1.0|  212|
+-----+-----+



In [ ]:
#split the data
train_df, test_df = df_transformed.randomSplit([0.75,0.25])

In [ ]:
train_df.count()

445

In [ ]:
test_df.count()

124

In [ ]:
train_df.groupBy('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0|  286|
|  1.0|  159|
+-----+-----+



In [ ]:
test_df.groupBy('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0|   71|
|  1.0|   53|
+-----+-----+



In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
model = LogisticRegression(labelCol='label')
model

LogisticRegression_0b7c6f976cca

In [ ]:
trained_model = model.fit(train_df)

In [ ]:
train_predictions = trained_model.evaluate(train_df).predictions
train_predictions.show(5)

/content/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(5007,[0,527,1288...|  0.0|[19.3129023981768...|[0.99999999590255...|       0.0|
|(5007,[0,548,1280...|  0.0|[19.4629305334752...|[0.99999999647339...|       0.0|
|(5007,[0,610,962,...|  0.0|[19.1695168115998...|[0.99999999527083...|       0.0|
|(5007,[1,480,1156...|  0.0|[19.4831064109445...|[0.99999999654383...|       0.0|
|(5007,[1,547,1155...|  0.0|[19.5406849867077...|[0.99999999673721...|       0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 5 rows



Predictions

In [ ]:
train_df_count_1 = train_df.filter(train_df['label'] == 1).count()
train_df_count_0 = train_df.filter(train_df['label'] == 0).count()
train_df_count_1, train_df_count_0

(159, 286)

In [ ]:
cp = train_predictions.filter(
train_predictions['label'] == 1).filter(
train_predictions['prediction'] == 1).select(
['label','prediction','probability'])
print("Correct predictions: ", cp.count())
accuracy = (cp.count()) /  train_df_count_1
print(f"Accuracy: {accuracy}\n")
cp.show(5,False)

Correct predictions:  159
Accuracy: 1.0

+-----+----------+-------------------------------------------+
|label|prediction|probability                                |
+-----+----------+-------------------------------------------+
|1.0  |1.0       |[4.1208969778968306E-10,0.9999999995879103]|
|1.0  |1.0       |[1.3650900443317923E-10,0.999999999863491] |
|1.0  |1.0       |[4.098880577394454E-10,0.999999999590112]  |
|1.0  |1.0       |[2.1731560340827983E-12,0.9999999999978268]|
|1.0  |1.0       |[2.1191520685333544E-11,0.9999999999788085]|
+-----+----------+-------------------------------------------+
only showing top 5 rows



In [ ]:
fp = train_predictions.filter(
train_predictions['label'] == 0).filter(
train_predictions['prediction'] == 1).select(
['label','prediction','probability'])
print("False positive: ", fp.count())
fp.show(5,False)

False positive:  0
+-----+----------+-----------+
|label|prediction|probability|
+-----+----------+-----------+
+-----+----------+-----------+



In [ ]:
fn = train_predictions.filter(
train_predictions['label'] == 1).filter(
train_predictions['prediction'] == 0).select(
['label','prediction','probability'])
print("False negative: ", fn.count())
fn.show(5,False)

False negative:  0
+-----+----------+-----------+
|label|prediction|probability|
+-----+----------+-----------+
+-----+----------+-----------+



In [ ]:
test_predictions = trained_model.evaluate(test_df).predictions
test_predictions.show(5, False)

/content/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+------------------------------------------------------------------------------------------------+-----+----------------------------------------+-----------------------------------------+----------+
|features                                                                                        |label|rawPrediction                           |probability                              |prediction|
+------------------------------------------------------------------------------------------------+-----+----------------------------------------+-----------------------------------------+----------+
|(5007,[0,900,1296,1757,2384,2891,3014,3855,4101,4930],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|1.0  |[4.491942141779461,-4.491942141779461]  |[0.9889251529280676,0.011074847071932403]|0.0       |
|(5007,[1,467,1157,1460,2354,2760,3171,3677,4274,4572],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0  |[11.219547251785531,-11.219547251785531]|[0.9999865906812176,1.340931878235363E-5]|0.0       |
|(500

In [ ]:
test_df_count_1 = test_df.filter(test_df['label'] == 1).count()
test_df_count_0 = test_df.filter(test_df['label'] == 0).count()
test_df_count_1, test_df_count_0

(53, 71)

Test Predictions

In [ ]:
cp = test_predictions.filter(
test_predictions['label'] == 1).filter(
test_predictions['prediction'] == 1).select(
['label','prediction','probability'])
print("Correct predictions: ", cp.count())
accuracy = (cp.count()) /  test_df_count_1
print(f"Accuracy: {accuracy}\n")
cp.show(5,False)

Correct predictions:  7
Accuracy: 0.1320754716981132

+-----+----------+------------------------------------------+
|label|prediction|probability                               |
+-----+----------+------------------------------------------+
|1.0  |1.0       |[0.02353455008753589,0.9764654499124641]  |
|1.0  |1.0       |[0.0051343226178753015,0.9948656773821247]|
|1.0  |1.0       |[0.3247547343685795,0.6752452656314205]   |
|1.0  |1.0       |[0.09172908641536762,0.9082709135846324]  |
|1.0  |1.0       |[0.3805473485631885,0.6194526514368115]   |
+-----+----------+------------------------------------------+
only showing top 5 rows



In [ ]:
fp = test_predictions.filter(
test_predictions['label'] == 0).filter(
test_predictions['prediction'] == 1).select(
['label','prediction','probability'])
print("False positive: ", fp.count())
fp.show(5,False)

False positive:  0
+-----+----------+-----------+
|label|prediction|probability|
+-----+----------+-----------+
+-----+----------+-----------+



In [ ]:
fn = test_predictions.filter(
test_predictions['label'] == 1).filter(
test_predictions['prediction'] == 0).select(
['label','prediction','probability'])
print("False negative: ", fn.count())
fn.show(5,False)

False negative:  46
+-----+----------+-----------------------------------------+
|label|prediction|probability                              |
+-----+----------+-----------------------------------------+
|1.0  |0.0       |[0.9889251529280676,0.011074847071932403]|
|1.0  |0.0       |[0.9990342126272204,9.657873727796362E-4]|
|1.0  |0.0       |[0.9889053230526537,0.011094676947346338]|
|1.0  |0.0       |[0.9163256993898229,0.08367430061017711] |
|1.0  |0.0       |[0.5879139263479234,0.4120860736520766]  |
+-----+----------+-----------------------------------------+
only showing top 5 rows



In [ ]:
end_time <- sys.time()
end_time-start-time

NameError: ignored